In [1]:
# ============================================
# DATA PREPROCESSING
# ============================================
# Tujuan: Menyiapkan data untuk machine learning
# Steps:
# 1. Handle missing values & data type issues
# 2. Feature engineering (tenure groups, service count)
# 3. Encoding categorical variables
# 4. Feature scaling
# 5. Train-test split
# ============================================

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import joblib


In [3]:
# ============================================
# Load Dataset
# ============================================

df = pd.read_csv(r'D:\churn-app\data\raw\TelcoCustomerChurn.csv')
print(f"Shape: {df.shape}")
print(f"Churn rate: {df['Churn'].value_counts(normalize=True)}")
df.head()


Shape: (7043, 21)
Churn rate: Churn
No     0.73463
Yes    0.26537
Name: proportion, dtype: float64


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
# ============================================
# Cek Data Types & Missing Values
# ============================================
# Perhatikan: TotalCharges bertipe object (string)
# Ini karena ada whitespace di beberapa row

print(df.info())
print("\n" + "="*50)
print("\nMissing values:")
print(df.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [6]:
# ============================================
# Handle TotalCharges
# ============================================
# Problem: TotalCharges adalah string, ada whitespace
# Solution: Convert ke numeric, fill missing dengan median

print(f"TotalCharges type: {df['TotalCharges'].dtype}")

# Convert to numeric (whitespace jadi NaN)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Check missing values after conversion
print(f"\nMissing TotalCharges: {df['TotalCharges'].isnull().sum()}")

# Fill missing dengan median
df['TotalCharges'] = df['TotalCharges'].fillna(df['TotalCharges'].median())

print(f"After filling: {df['TotalCharges'].isnull().sum()}")


TotalCharges type: float64

Missing TotalCharges: 0
After filling: 0


In [7]:
# ============================================
# Drop customerID
# ============================================
# customerID tidak berguna untuk prediksi
# Hanya identifier unik per customer

df.drop('customerID', axis=1, inplace=True)

print(f"Shape after dropping customerID: {df.shape}")
df.head()


Shape after dropping customerID: (7043, 20)


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [8]:
# ============================================
# Feature Engineering
# ============================================
# Membuat fitur baru yang mungkin lebih informatif:
# 1. tenure_group: Kategorisasi tenure (0-1yr, 1-2yr, dll)
# 2. avg_charge_per_month: Total charges / tenure
# 3. num_services: Jumlah layanan yang digunakan

# 1. Tenure groups
df['tenure_group'] = pd.cut(
    df['tenure'], 
    bins=[0, 12, 24, 48, 72], 
    labels=['0-1yr', '1-2yr', '2-4yr', '4+yr']
)

# 2. Average charge per month
df['avg_charge_per_month'] = df['TotalCharges'] / (df['tenure'] + 1)

# 3. Service count (berapa banyak services yang dipakai)
service_cols = ['PhoneService', 'InternetService', 'OnlineSecurity',
                'OnlineBackup', 'DeviceProtection', 'TechSupport',
                'StreamingTV', 'StreamingMovies']

df['num_services'] = 0
for col in service_cols:
    if col == 'InternetService':
        df['num_services'] += (df[col] != 'No').astype(int)
    else:
        df['num_services'] += (df[col] == 'Yes').astype(int)

print(f"New features created!")
print(f"\nTenure groups distribution:\n{df['tenure_group'].value_counts()}")
print(f"\nAverage services per customer: {df['num_services'].mean():.2f}")


New features created!

Tenure groups distribution:
tenure_group
4+yr     2239
0-1yr    2175
2-4yr    1594
1-2yr    1024
Name: count, dtype: int64

Average services per customer: 3.72


In [9]:
# ============================================
# Encode Target Variable (Churn)
# ============================================
# Convert Yes/No ke 1/0 untuk modeling

print("Before encoding:")
print(df['Churn'].value_counts())

df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})

print("\nAfter encoding:")
print(df['Churn'].value_counts())


Before encoding:
Churn
No     5174
Yes    1869
Name: count, dtype: int64

After encoding:
Churn
0    5174
1    1869
Name: count, dtype: int64


In [10]:
# ============================================
# Binary Encoding (Yes/No columns)
# ============================================
# Kolom dengan nilai Yes/No di-convert ke 1/0

binary_cols = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']

for col in binary_cols:
    df[col] = df[col].map({'Yes': 1, 'No': 0})

print(f"Binary columns encoded: {binary_cols}")


Binary columns encoded: ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']


In [11]:
# ============================================
# Cek Categorical Columns untuk One-Hot Encoding
# ============================================
# Kolom dengan >2 unique values perlu one-hot encoding

categorical_cols = ['gender', 'MultipleLines', 'InternetService', 
                    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                    'TechSupport', 'StreamingTV', 'StreamingMovies',
                    'Contract', 'PaymentMethod', 'tenure_group']

# Check unique values
for col in categorical_cols:
    print(f"{col}: {df[col].unique()}")


gender: ['Female' 'Male']
MultipleLines: ['No phone service' 'No' 'Yes']
InternetService: ['DSL' 'Fiber optic' 'No']
OnlineSecurity: ['No' 'Yes' 'No internet service']
OnlineBackup: ['Yes' 'No' 'No internet service']
DeviceProtection: ['No' 'Yes' 'No internet service']
TechSupport: ['No' 'Yes' 'No internet service']
StreamingTV: ['No' 'Yes' 'No internet service']
StreamingMovies: ['No' 'Yes' 'No internet service']
Contract: ['Month-to-month' 'One year' 'Two year']
PaymentMethod: ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
tenure_group: ['0-1yr', '2-4yr', '1-2yr', '4+yr', NaN]
Categories (4, object): ['0-1yr' < '1-2yr' < '2-4yr' < '4+yr']


In [13]:
# Cell 10: One-hot encoding
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

print(f"\nShape before encoding: {df.shape}")
print(f"Shape after encoding: {df_encoded.shape}")
print(f"\nNew columns: {df_encoded.shape[1] - df.shape[1]} added")

df_encoded.head()


Shape before encoding: (7043, 23)
Shape after encoding: (7043, 36)

New columns: 13 added


,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,avg_charge_per_month,...,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1-2yr,tenure_group_2-4yr,tenure_group_4+yr
0,0,1,0,1,0,1,29.85,29.85,0,14.925000,...,False,False,False,False,False,True,False,False,False,False
1,0,0,0,34,1,0,56.95,1889.50,0,53.985714,...,False,False,True,False,False,False,True,False,True,False
2,0,0,0,2,1,1,53.85,108.15,1,36.050000,...,False,False,False,False,False,False,True,False,False,False
3,0,0,0,45,0,0,42.30,1840.75,0,40.016304,...,False,False,True,False,False,False,False,False,True,False
4,0,0,0,2,1,1,70.70,151.65,1,50.550000,...,False,False,False,False,False,True,False,False,False,False


In [14]:
# Separate X and y
X = df_encoded.drop('Churn', axis=1)
y = df_encoded['Churn']

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nChurn distribution:")
print(y.value_counts(normalize=True))

Features shape: (7043, 35)
Target shape: (7043,)

Churn distribution:
Churn
0    0.73463
1    0.26537
Name: proportion, dtype: float64


In [15]:
# Split data (stratified karena imbalanced)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print(f"Train set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print(f"\nTrain churn rate: {y_train.mean():.2%}")
print(f"Test churn rate: {y_test.mean():.2%}")

Train set: (5634, 35)
Test set: (1409, 35)

Train churn rate: 26.54%
Test churn rate: 26.54%


In [16]:
# Identify numerical columns
numerical_cols = ['tenure', 'MonthlyCharges', 'TotalCharges', 'avg_charge_per_month', 'num_services']

# Initialize scaler
scaler = StandardScaler()

# Fit on train, transform both
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

print("✅ Features scaled!")
print(f"\nSample scaled values:")
print(X_train[numerical_cols].head())

✅ Features scaled!

Sample scaled values:
        tenure  MonthlyCharges  TotalCharges  avg_charge_per_month  \
3738  0.102371       -0.521976     -0.263289             -0.236288   
3151 -0.711743        0.337478     -0.504814              0.182960   
4860 -0.793155       -0.809013     -0.751213             -0.322843   
3867 -0.263980        0.284384     -0.173699              0.159364   
3810 -1.281624       -0.676279     -0.990851             -0.660445   

      num_services  
3738      0.121882  
3151     -0.353002  
4860      0.121882  
3867      1.071649  
3810     -0.827886  


In [17]:
# Save processed data
X_train.to_csv(r'D:\churn-app\data\processed\X_train.csv', index=False)
X_test.to_csv(r'D:\churn-app\data\processed\X_test.csv', index=False)
y_train.to_csv(r'D:\churn-app\data\processed\y_train.csv', index=False)
y_test.to_csv(r'D:\churn-app\data\processed\y_test.csv', index=False)

# Save scaler
joblib.dump(scaler, r'D:\churn-app\models\scaler.pkl')

print("✅ All data saved!")
print("\nSaved files:")
print("- X_train.csv")
print("- X_test.csv")
print("- y_train.csv")
print("- y_test.csv")
print("- scaler.pkl")

✅ All data saved!

Saved files:
- X_train.csv
- X_test.csv
- y_train.csv
- y_test.csv
- scaler.pkl


In [19]:
# Final summary
print("="*60)
print("DATA PREPROCESSING SUMMARY")
print("="*60)
print(f"\n1. Original data: {df.shape[0]} rows, {df.shape[1]} columns")
print(f"2. After encoding: {df_encoded.shape[0]} rows, {df_encoded.shape[1]} columns")
print(f"3. Features (X): {X.shape[1]} columns")
print(f"4. Train set: {X_train.shape[0]} samples ({len(y_train)/len(y)*100:.1f}%)")
print(f"5. Test set: {X_test.shape[0]} samples ({len(y_test)/len(y)*100:.1f}%)")
print(f"6. Churn rate (train): {y_train.mean():.2%}")
print(f"7. Churn rate (test): {y_test.mean():.2%}")

DATA PREPROCESSING SUMMARY

1. Original data: 7043 rows, 23 columns
2. After encoding: 7043 rows, 36 columns
3. Features (X): 35 columns
4. Train set: 5634 samples (80.0%)
5. Test set: 1409 samples (20.0%)
6. Churn rate (train): 26.54%
7. Churn rate (test): 26.54%


In [20]:
# Cell baru: Print all column names
print("Total features:", X_train.shape[1])
print("\nAll feature names:")
print(X_train.columns.tolist())

# Save ke file biar gampang copy
with open(r'D:\churn-app\outputs\feature_names.txt', 'w') as f:
    for col in X_train.columns:
        f.write(col + '\n')

print("\n✅ Feature names saved to outputs/feature_names.txt")

Total features: 35

All feature names:
['SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'PaperlessBilling', 'MonthlyCharges', 'TotalCharges', 'avg_charge_per_month', 'num_services', 'gender_Male', 'MultipleLines_No phone service', 'MultipleLines_Yes', 'InternetService_Fiber optic', 'InternetService_No', 'OnlineSecurity_No internet service', 'OnlineSecurity_Yes', 'OnlineBackup_No internet service', 'OnlineBackup_Yes', 'DeviceProtection_No internet service', 'DeviceProtection_Yes', 'TechSupport_No internet service', 'TechSupport_Yes', 'StreamingTV_No internet service', 'StreamingTV_Yes', 'StreamingMovies_No internet service', 'StreamingMovies_Yes', 'Contract_One year', 'Contract_Two year', 'PaymentMethod_Credit card (automatic)', 'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check', 'tenure_group_1-2yr', 'tenure_group_2-4yr', 'tenure_group_4+yr']

✅ Feature names saved to outputs/feature_names.txt
